In [1]:
# !pip install MetaTrader5

import MetaTrader5 as mt5
from datetime import datetime, timedelta
import pandas as pd
import time
from tqdm.notebook import tqdm

if not mt5.initialize():
    mt5.shutdown()
mt5.terminal_info()

In [4]:
data = [_.split('\t') for _ in """

	 ### paste here ####



USDJPY	110.255	0.04	0.1321125
USDJPY	109.128	0.04	0.1321125
GBPUSD	1.40433	0.02	0.0023825
GBPUSD	1.4221	0.02	0.0023825
GBPJPY	152.188	0.02	0.2721845
GBPCHF	1.2846	0.02	0.0021375
GBPCHF	1.26238	0.02	0.0021375
GBPAUD	1.84034	0.02	0.0032625
		#N/A	-
GBPAUD	1.80413	0.02	0.0032625
EURUSD	1.20138	0.03	0.0018245
EURNZD	1.67116	0.02	0.0031155
EURJPY	134.022	0.03	0.202949
EURJPY	132.613	0.03	0.202949
EURCAD	1.46578	0.03	0.002112
EURAUD	1.58315	0.02	0.0027895
EURAUD	1.56417	0.02	0.0027895
CHFJPY	122.447	0.04	0.156496
CHFJPY	121.472	0.04	0.156496
CHFJPY	118.848	0.04	0.156496




""".split('\n') if _.split('\t')[0] != '']

data = [
    [
        _[0]
        , float(_[1])
        , round(float(_[2]), 2)
        , round(float(_[3]), 6)
    ] for _ in data
]

print(data)

# for inst in data (unique),
# get current price
# if pending above, sell
# if pending below, buy

p = []
for i in tqdm(set([_[0] for _ in data])):
    a = mt5.symbol_info_tick(i)
    p.append([i, a])

p = [
    [
        _[0], 
        round((_[1].bid + _[1].ask) / 2, 6)
    ] for _ in p
]

print(p)

results = []
for _ in data:
    if _[1] > [x[1] for x in p if x[0] == _[0]][0]:
        print('sell', _, )
        
        r = {
            "action": mt5.TRADE_ACTION_PENDING,
            "symbol": _[0],
            "volume": _[2],
            "type": mt5.ORDER_TYPE_SELL_LIMIT,
            "price": _[1],
            "sl": _[1] + _[3],
            "tp": _[1] - (_[3]*3),
            "deviation": 10,
            "magic": 9,
            "comment": "sent by python",
            "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result = mt5.order_send(r)  
    else:
        print('buy', _, )
        r = {
            "action": mt5.TRADE_ACTION_PENDING,
            "symbol": _[0],
            "volume": _[2],
            "type": mt5.ORDER_TYPE_BUY_LIMIT,
            "price": _[1],
            "sl": _[1] - _[3],  ############ this is getting mixed up for shorts
            "tp": _[1] + (_[3]*3),
            "deviation": 10,
            "magic": 9,
            "comment": "sent by python",
            "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result = mt5.order_send(r)  
    results.append(result)

results

[['USDJPY', 110.255, 0.04, 0.132112],
 ['USDJPY', 109.128, 0.04, 0.132112],
 ['GBPUSD', 1.40433, 0.02, 0.002383],
 ['GBPUSD', 1.4221, 0.02, 0.002383],
 ['GBPJPY', 152.188, 0.02, 0.272184],
 ['GBPCHF', 1.2846, 0.02, 0.002138],
 ['GBPCHF', 1.26238, 0.02, 0.002138],
 ['GBPAUD', 1.84034, 0.02, 0.003263],
 ['GBPAUD', 1.80413, 0.02, 0.003263],
 ['EURUSD', 1.20138, 0.03, 0.001824],
 ['EURNZD', 1.67116, 0.02, 0.003116],
 ['EURJPY', 134.022, 0.03, 0.202949],
 ['EURJPY', 132.613, 0.03, 0.202949],
 ['EURCAD', 1.46578, 0.03, 0.002112],
 ['EURAUD', 1.58315, 0.02, 0.002789],
 ['EURAUD', 1.56417, 0.02, 0.002789],
 ['CHFJPY', 122.447, 0.04, 0.156496],
 ['CHFJPY', 121.472, 0.04, 0.156496],
 ['CHFJPY', 118.848, 0.04, 0.156496]]